# **Image Processing: Eye Fundus**
https://en.wikipedia.org/wiki/Fundus_(eye)

## Extracting Data

In [ ]:
!unzip Eye.zip

## Necessary Imports and Helper Code

Imports and constants

In [ ]:
import cv2
import numpy as np
import os

import matplotlib.pyplot as plt

RGB = cv2.COLOR_BGR2RGB
YCrCb = cv2.COLOR_BGR2YCR_CB
R, G, B = 0, 1, 2
Y, Cr, Cb = 0, 1, 2
color_space_idx = {
    RGB: {0: 'Red', 1: 'Green', 2: 'Blue'},
    YCrCb: {0: 'Yellow', 1: 'Red diff chroma', 2: 'Blue diff chroma'}
}

Helper code - extract and compare planes

In [ ]:
# Can pass in different enhancement functions written to enhance a channel to this function and compare them
# visually
def compare_enhancements(color_space, channel_idx, display_hist=False, *enhancements):
    fig = plt.figure(figsize=(20, 80))
    rows = 15
    h = 1 if display_hist==True else 0
    columns = len(enhancements) + 1
    for i, entry in enumerate(os.scandir('Eye')):
        if entry.is_file():
            image = cv2.cvtColor(cv2.imread(entry.path, 1), color_space)
            if h == 1:
                fig.add_subplot(rows, columns, columns*i+h)
                plt.title('Histogram: '+ entry.name)
                plt.hist(image[:,:,channel_idx].ravel(),256,[0,256])
            for idx, enhancement in enumerate(enhancements):
                fig.add_subplot(rows, columns, columns*i+1+h+idx)
                if idx == 0:
                    plt.title(enhancement.__name__ + ' ' + ' ' + entry.name)
                else:
                    plt.title(enhancement.__name__)
                plt.imshow(enhancement(image[:,:,channel_idx]), cmap='gray', vmin = 0, vmax = 255)


# Function to extract(display) planes according to colorspaces RGB and YCrCb
def display_planes(colorspace):
    fig = plt.figure(figsize=(20, 80))
    rows = 15
    columns = 4
    for i, entry in enumerate(os.scandir('Eye')):
        if entry.is_file():
            image = cv2.cvtColor(cv2.imread(entry.path, 1), colorspace)
            fig.add_subplot(rows, columns, 4*i+1)
            plt.title('Original: '+ entry.name)
            plt.imshow(image)
            fig.add_subplot(rows, columns, 4*i+2)
            plt.title(f'{color_space_idx[colorspace][0]} Channel: '+ entry.name)
            plt.imshow(image[:,:,0],cmap='gray', vmin = 0, vmax = 255)
            fig.add_subplot(rows, columns, 4*i+3)
            plt.title(f'{color_space_idx[colorspace][1]} Channel: '+ entry.name)
            plt.imshow(image[:,:,1],cmap='gray', vmin = 0, vmax = 255)
            fig.add_subplot(rows, columns, 4*i+4)
            plt.title(f'{color_space_idx[colorspace][2]} Channel: '+ entry.name)
            plt.imshow(image[:,:,2],cmap='gray', vmin = 0, vmax = 255)

Helper code - transformations and enhancements

In [ ]:
# Transformations and Enhancements

def identity(image):
    return image

def inverse_image(image):
    return 1 - image

def log_image(image):
    c = 255 / np.log(1 + np.max(image))
    log_image = c * (np.log(image + 1))
    log_image = np.array(log_image, dtype = np.uint8)
    return log_image

def inv_log_image(image):
    inv_log_image = 1 - log_image(image)
    return inv_log_image

# Different gamma parents require different methods! Else the compare_enhancements method becomes ugly
def gamma_4_image(image):
    c = 1
    gamma = 4
    power_image = c * np.array(255*(image/255)**gamma,dtype='uint8')
    power_image = np.array(power_image, dtype = np.uint8)
    return power_image

def gamma_3_image(image):
    c = 1
    gamma = 3
    power_image = c * np.array(255*(image/255)**gamma,dtype='uint8')
    power_image = np.array(power_image, dtype = np.uint8)
    return power_image

def gamma_2_image(image):
    c = 1
    gamma = 2
    power_image = c * np.array(255*(image/255)**gamma,dtype='uint8')
    power_image = np.array(power_image, dtype = np.uint8)
    return power_image

# Different gamma parents require different methods! Else the compare_enhancements method becomes ugly
def gamma_p5_image(image):
    c = 1
    gamma = 0.5
    power_image = c * np.array(255*(image/255)**gamma,dtype='uint8')
    power_image = np.array(power_image, dtype = np.uint8)
    return power_image

def gamma_p4_image(image):
    c = 1
    gamma = 0.4
    power_image = c * np.array(255*(image/255)**gamma,dtype='uint8')
    power_image = np.array(power_image, dtype = np.uint8)
    return power_image

def gamma_p3_image(image):
    c = 1
    gamma = 0.3
    power_image = c * np.array(255*(image/255)**gamma,dtype='uint8')
    power_image = np.array(power_image, dtype = np.uint8)
    return power_image

def gamma_p2_image(image):
    c = 1
    gamma = 0.2
    power_image = c * np.array(255*(image/255)**gamma,dtype='uint8')
    power_image = np.array(power_image, dtype = np.uint8)
    return power_image

def hist_equalized_image(image):
    he_image = cv2.equalizeHist(image)
    return he_image

def contrast_stretch_image(image):
    def pixelVal(pix, r1, s1, r2, s2):
        if (0 <= pix and pix <= r1):
            return (s1 / r1)*pix
        elif (r1 < pix and pix <= r2):
            return ((s2 - s1)/(r2 - r1)) * (pix - r1) + s1
        else:
            return ((255 - s2)/(255 - r2)) * (pix - r2) + s2
    r1 = 70
    s1 = 45
    r2 = 140
    s2 = 210
    pixelVal_vec = np.vectorize(pixelVal)
    contrast_stretched = pixelVal_vec(image, r1, s1, r2, s2)
    return contrast_stretched

def intensity_sliced_image(image):
    def pixelVal(pix, r1, r2):
        if (0 <= pix and pix <= r1):
            return pix
        elif (r1 < pix and pix <= r2):
            return 100
        else:
            return pix
    r1 = 70
    r2 = 140
    pixelVal_vec = np.vectorize(pixelVal)
    intensitySliced_stretched_image = pixelVal_vec(image, r1, r2)
    return intensitySliced_stretched_image

def sharpening_kernel1(image):
    kernel1 = np.array([[0, -1, 0],
                        [-1, 5, -1],
                        [0, -1, 0]])
    sharp_img = cv2.filter2D(src=image, ddepth=-1, kernel=kernel1)
    return sharp_img

def sharpening_kernel2(image):
    kernel2 = np.array([[-1, -1, -1],
                        [-1, 9, -1],
                        [-1, -1, -1]])
    sharp_img = cv2.filter2D(src=image, ddepth=-1, kernel=kernel2)
    return sharp_img

def combination1(image):
    image1 = log_image(image)
    comb_image = sharpening_kernel2(image1)
    return comb_image

## Extract planes - Red(R), Green(G), Blue(B)

In [ ]:
display_planes(RGB)

### Enhancing Red Channel
**This channel is too bright to infer useful visual info. So we try different enhancement methods - redistributing pixel values from bright pixel ranges to darker ranges and pick the one that give the best result visually. Methods: histogram equalization / power law transformation / contrast stretching / inverse log transformation and pick the best among them.**

In [ ]:
compare_enhancements(RGB, 0, False, identity,
                     hist_equalized_image,
                     gamma_3_image,
                     contrast_stretch_image,
                     inv_log_image
                     )

**We can see that histogram equilization gives the best enhancement for most images**

In [ ]:
#select function that gave the best enhancement
red_enhancement = hist_equalized_image

### Enhancing Green Channel
**This channel has most information about nerves so we try different sharpening methods suited for this scenario and pick the one that give the best result visually. Methods: sharpening_kernel1 / sharpening_kernel2**

In [ ]:
compare_enhancements(RGB, 1, False, identity, sharpening_kernel1, sharpening_kernel2)

**We can see that sharpening_kernel2 gives the best enhancement overall**

In [ ]:
#select function that gave the best enhancement
green_enhancement = sharpening_kernel2

### Enhancing Blue Channel
**This channel are very dark to infer useful info. So we try different enhancement methods  redistributing pixel values from dark pixel ranges to brighter ranges and pick the one that give the best result visually. Methods: power law transformation / contrast stretching / log transformation or some combinaton and pick the best among them.**

In [ ]:
compare_enhancements(RGB, 2, False, identity,
                     gamma_p3_image,
                     combination1,
                     contrast_stretch_image,
                     log_image)

**We can see that the enhancements are not really great neverthless combination1 - log_image + sharpening_kernel2 gives the best enhancement overall**

In [ ]:
# select function that gave the best enhancement
blue_enhancement = combination1

### Try Combinations of enhanced Channels

#### One channel enhanced

In [ ]:
fig = plt.figure(figsize=(20, 80))
rows = 15
columns = 4
for i, entry in enumerate(os.scandir('Eye')):
    if entry.is_file():
        image = cv2.cvtColor(cv2.imread(entry.path, 1), cv2.COLOR_BGR2RGB)

        # original channels
        or_channel = image[:,:,0]
        og_channel = image[:,:,1]
        ob_channel = image[:,:,2]
        # enhanced channels
        er_channel = red_enhancement(image[:,:,0])
        eg_channel = green_enhancement(image[:,:,1])
        eb_channel = blue_enhancement(image[:,:,2])

        # Plot combinations
        # original image
        fig.add_subplot(rows, columns, columns*i+1)
        plt.title('Original: '+ entry.name)
        plt.imshow(image)

        # image with one enhanced channels
        fig.add_subplot(rows, columns, columns*i+2)
        plt.title('EnhancedRed: ')
        plt.imshow(cv2.merge((er_channel, og_channel, ob_channel)))
        fig.add_subplot(rows, columns, columns*i+3)
        plt.title('EnhancedGreen: '+ entry.name)
        plt.imshow(cv2.merge((or_channel, eg_channel, ob_channel)))
        fig.add_subplot(rows, columns, columns*i+4)
        plt.title('EnhancedBlue: '+ entry.name)
        plt.imshow(cv2.merge((or_channel, og_channel, eb_channel)))

#### Two channels enhanced

In [ ]:
fig = plt.figure(figsize=(20, 80))
rows = 15
columns = 5
for i, entry in enumerate(os.scandir('Eye')):
    if entry.is_file():
        image = cv2.cvtColor(cv2.imread(entry.path, 1), cv2.COLOR_BGR2RGB)

        # original channels
        or_channel = image[:,:,0]
        og_channel = image[:,:,1]
        ob_channel = image[:,:,2]
        # enhanced channels
        er_channel = red_enhancement(image[:,:,0])
        eg_channel = green_enhancement(image[:,:,1])
        eb_channel = blue_enhancement(image[:,:,2])

        # Plot combinations
        # original image
        fig.add_subplot(rows, columns, columns*i+1)
        plt.title('Original: '+ entry.name)
        plt.imshow(image)

        # image with two enhanced channels
        fig.add_subplot(rows, columns, columns*i+2)
        plt.title('EnhancedRedGreen: ')
        plt.imshow(cv2.merge((er_channel, eg_channel, ob_channel)))
        fig.add_subplot(rows, columns, columns*i+3)
        plt.title('EnhancedGreenBlue Channel: ')
        plt.imshow(cv2.merge((or_channel, eg_channel, eb_channel)))
        fig.add_subplot(rows, columns, columns*i+4)
        plt.title('EnhancedBlueRed Channel: ')
        plt.imshow(cv2.merge((er_channel, og_channel, eb_channel)))

        # image with all channels enhanced
        fig.add_subplot(rows, columns, columns*i+5)
        plt.title('Enhanced RGB Channel: ')
        plt.imshow(cv2.merge((er_channel, eg_channel, eb_channel)))

#### Comparison between 'best' channels

In [ ]:
fig = plt.figure(figsize=(20, 80))
rows = 15
columns = 4
for i, entry in enumerate(os.scandir('Eye')):
    if entry.is_file():
        image = cv2.cvtColor(cv2.imread(entry.path, 1), cv2.COLOR_BGR2RGB)

        # original channels
        or_channel = image[:,:,0]
        og_channel = image[:,:,1]
        ob_channel = image[:,:,2]
        # enhanced channels
        er_channel = red_enhancement(image[:,:,0])
        eg_channel = green_enhancement(image[:,:,1])
        eb_channel = blue_enhancement(image[:,:,2])

        # Plot combinations
        # original image
        fig.add_subplot(rows, columns, columns*i+1)
        plt.title('Original: '+ entry.name)
        plt.imshow(image)

        # image with one enhanced channels
        fig.add_subplot(rows, columns, columns*i+2)
        plt.title('EnhancedGreen: ')
        plt.imshow(cv2.merge((or_channel, eg_channel, ob_channel)))

        # image with two enhanced channels
        fig.add_subplot(rows, columns, columns*i+3)
        plt.title('EnhancedRedGreen: ')
        plt.imshow(cv2.merge((er_channel, eg_channel, ob_channel)))

        # image with all channels enhanced
        fig.add_subplot(rows, columns, columns*i+4)
        plt.title('Enhanced RGB Channel: ')
        plt.imshow(cv2.merge((er_channel, eg_channel, eb_channel)))

### Inference
**Some of these are very close, but overall we can see that the combination with enhancement on both red and green retains and enhances most information : The nerves and the bright spot where they join are visually more perceivable and crisp**

## Extract planes - Luma/brightness(Y), Chroma components(Cb),(Cr)

In [ ]:
display_planes(YCrCb)

### Enhancing Y Channel
**This channel has most information about nerves so we try different sharpening methods and pick the one that give the best result visually. Methods: sharpening_kernel1 / sharpening_kernel2**

In [ ]:
compare_enhancements(YCrCb, 0, False, identity,
                     sharpening_kernel1,
                     sharpening_kernel2)

**We can see that overall, the best enhancement for the images is from applying sharpening_kernel2**

In [ ]:
# select function that gave the best enhancement
y_enhancement = sharpening_kernel2

### Enhancing Cr Channel
**This channel is somewhat bright to infer useful info. So we try different enhancement methods and pick the one that give the best result visually. Methods: histogram equalization / power law transformation / inverse log transformation and pick the best among them.**

In [ ]:
compare_enhancements(YCrCb, 1, False, identity,
                     hist_equalized_image,
                     gamma_2_image,
                     contrast_stretch_image,
                     inv_log_image)

**We can see that power transformation with gamma 2 gives good enhancement**

In [ ]:
# select function that gave the best enhancement
cr_enhancement = gamma_2_image

### Enhancing Cb Channel
**This channel is somewhat dark to infer useful info. So we try different enhancement methods and pick the one that give the best result visually. Methods: histogram equalization / power law transformation / log transformation or some combinaton and pick the best among them.**

In [ ]:
compare_enhancements(YCrCb, 2, False, identity,
                     hist_equalized_image,
                     gamma_p5_image,
                     gamma_p2_image,
                     log_image)

**We can see that the histogram equalized image gives good enhancement as it appears to make the veins and the darker regions of image more pronounced**

In [ ]:
cb_enhancement = hist_equalized_image

### Try Combinations of enhanced Channels

**One channel enhanced**

In [ ]:
fig = plt.figure(figsize=(20, 80))
rows = 15
columns = 4
for i, entry in enumerate(os.scandir('Eye')):
    if entry.is_file():
        image = cv2.cvtColor(cv2.imread(entry.path, 1), YCrCb)

        # original channels
        oy_channel = image[:,:,0]
        ocr_channel = image[:,:,1]
        ocb_channel = image[:,:,2]
        #emhanced channels
        ey_channel = y_enhancement(image[:,:,0])
        ecr_channel = cr_enhancement(image[:,:,1])
        ecb_channel = cb_enhancement(image[:,:,2])

        # Plot combinations
        # original image
        fig.add_subplot(rows, columns, columns*i+1)
        plt.title('Original: '+ entry.name)
        plt.imshow(image)

        # image with one enhanced channels
        fig.add_subplot(rows, columns, columns*i+2)
        plt.title('EnhancedY: ')
        plt.imshow(cv2.merge((ey_channel, ocr_channel, ocb_channel)))
        fig.add_subplot(rows, columns, columns*i+3)
        plt.title('EnhancedCr: ')
        plt.imshow(cv2.merge((oy_channel, ecr_channel, ocb_channel)))
        fig.add_subplot(rows, columns, columns*i+4)
        plt.title('EnhancedCb: ')
        plt.imshow(cv2.merge((oy_channel, ocr_channel, ecb_channel)))

**Two channels enhanced**

In [ ]:
fig = plt.figure(figsize=(20, 80))
rows = 15
columns = 5
for i, entry in enumerate(os.scandir('Eye')):
    if entry.is_file():
        image = cv2.cvtColor(cv2.imread(entry.path, 1), YCrCb)

        # original channels
        oy_channel = image[:,:,0]
        ocr_channel = image[:,:,1]
        ocb_channel = image[:,:,2]
        #emhanced channels
        ey_channel = y_enhancement(image[:,:,0])
        ecr_channel = cr_enhancement(image[:,:,1])
        ecb_channel = cb_enhancement(image[:,:,2])

        # Plot combinations
        # original image
        fig.add_subplot(rows, columns, columns*i+1)
        plt.title('Original: '+ entry.name)
        plt.imshow(image)

        # image with two enhanced channels
        fig.add_subplot(rows, columns, columns*i+2)
        plt.title('EnhancedYCr: ')
        plt.imshow(cv2.merge((ey_channel, ecr_channel, ocb_channel)))
        fig.add_subplot(rows, columns, columns*i+3)
        plt.title('EnhancedCrCb Channel: ')
        plt.imshow(cv2.merge((oy_channel, ecr_channel, ecb_channel)))
        fig.add_subplot(rows, columns, columns*i+4)
        plt.title('EnhancedYCb Channel: ')
        plt.imshow(cv2.merge((ey_channel, ocr_channel, ecb_channel)))

        # image with all channels enhanced
        fig.add_subplot(rows, columns, columns*i+5)
        plt.title('Enhanced YCrCb Channel: ')
        plt.imshow(cv2.merge((ey_channel, ecr_channel, ecb_channel)))

#### Inference
**From the above observations, in the YCrCb color space, more information seems to be enhanced and retained when all the channels are enhanced, but in somecases other combinations seem to enhance more useful information**

### Results

As seen above,

For RGB color space enhancing red and green channels worked most. This is because the green channel had the most information about the nerves (enhanced by sharpening to make blood vessels more sharp) and the red channel added some more information about the nerves and the bright spot were the nerves meet (enhanced by histogram equilization to make the brighter spot and vessels more distinct). The blue channel did not contain much relevant information and most of the intensities of this channel were very low to actually discern anything even after combinations of enhancement techniques.

For the YCrCb color space, it is difficult to comprehend regions of interest just from visualizing the originals. After extracting each planes, we see that the Y channel (enhanced by sharpening to sharpen the blood vessels) has the most information since it is the luma component and corresponds to the intensity. The Cr and Cb channels were enhanced by using power law transformation (gamma=2) and histogram equalization respectively to try to distinguish region of interest from its surroundings. From the combinations visually, it seems that all the enhanced channels are required to retain and enhance blood vessels and other distinctive regions in the image.